# Comparison of multicultural cities

## Introduction: Description & Disscusion of the Background

New York City is one of the large cities in USA, it's also the most densely populated major city in USA. The city constitutes the premier gateway for immigration to the USA. There are more than 800 languages are spoken in New York City. It's one of the most multicultural cities in the world.
Toronto is Canada's largest city, the fourth largest in North American, and home to a diverse population. The diverse population of Toronto reflects its role as an important destination for immigrants to Canada. Over 160 languages are spoken in the city. Toronto is also one of the most multicultural cities in the world.
Los Angeles is one of the most stylish cities in the world. Almost 100 language speaking people reside at Los Angeles and hence every style, dress and cuisines of this city is different. Los Angeles is a welcoming place for immigrants.It is also one of the most multicultural cities in the world.

New York, Toronto and Los Angeles are all multicultural cities. They attract thousands of people from all over the world to live in. While the cities have some similarities and may be attractive to various people depending on lifestyles and preferences. In this project we will try to compare these three multicultural cities to see how similar they are. There are so many ways to compare cities. In this project we would like to use foursquare data to compare venues (i.e. density, distribution, categories of venues) in these cities and their clustered neighbourhoods. We will use our data science powers to determine if the cities are similar or dissimilar in multiple ways. 

## Data Description

Based on definition of our problem, factors that will influence our decision are:
<br>
•	Population of the neighbourhood/city
<br>
•	Density of neighborhood/city
<br>
•	How many categories of venues in the neighborhood/city?
<br>
•	Density of venues in the neighborhood/city?
<br>
We can use these factors to determine if cities are similar to each other.

To consider the problem we can list the data as below:
<br>
I use https://geo.nyu.edu/catalog/nyu_2451_34572 data to get neighborhoods as well as the the latitude and logitude coordinates of each neighborhood of New York City. We can find Toronto's neibourhoods and the latitude and logitude coordinates of each neighborhood from wikipedia https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. A csv file contians latitude and logitude information of Toronto. Los Angeles neibourhoods information are available on SOCR Data LA Neighborhoods: Datahttp://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_LA_Neighborhoods_Data#SOCR_Data_-_Los_Angeles_City_Neighborhoods_Data.

I used Forsquare API to get the most common venues of given neibourhoods of three cities.






In [ ]:
!conda install -c conda-forge geopy --yes

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


vincent-0.4.4        | 28 KB     | #################################

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.
